### Characterising pixel response of BFS-U3-200S6M Sensor 📸

We want to model the non-linear response per pixel to incoming photons (of various ${\lambda}$ and across intensities).

There are multiple sources of noise that vary with wavelength (e.g. flicker or 1/f noise) and incoming intensity (e.g. shot noise) - 
to simplify analysis we will just fit a curve to each of these pixel responses outlined by:
https://www1.cs.columbia.edu/CAVE/publications/pdfs/Grossberg_PAMI04.pdf 

Lets try a gamma curve (bunch of lit refs that follow):
${f(E) = \alpha + \beta E^\gamma}$

Useful when performing phase retrieval (can only retrieve as good as your model represents)
(probably over-kill but this is an interesting task regardless)

Using 635nm laser source 🔴

In [1]:
import numpy as np
import matplotlib.pyplot as plt


### BFS Detector
CCD (Charge Coupled Device) sensors can be descirbed by a single value for readout noise because all charge is transported across the chip and read at a single corner of the array.

CMOS (Complementrary Metal-Oxide Semiconductor) sensors however have individual amplifier circuits per pixel, with each pixel/photosite read individually - so need per pixel read out noise to descibe these. **The BFS is a CMOS** detector

Shutters closed/0s exposure will give you readout noise

In [ ]:
# Import data


In [ ]:
# Fave optimiser to fit curve?


In [ ]:
# For a sweep of input, uniform intensities - measure the difference between pixels (flat field)